In [69]:
import os
# import easyocr
import cv2
import numpy as np
import requests
import json
import base64
import openai
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

In [70]:
# ✅ 폴더 경로 설정
save_folder = "download_images"
cropped_folder = "cropped_images"
text_folder = "ocr_texts"

In [71]:
def split_vertical_with_overlap(image_path, output_folder, crop_height=800, overlap=100):
    """
    긴 이미지를 일정한 높이로 나누되, 일정 부분을 겹쳐서 자르는 함수
    - image_path: 원본 이미지 경로
    - output_folder: 저장할 폴더
    - crop_height: 자를 높이 크기 (기본값: 800px)
    - overlap: 다음 이미지와 겹치는 부분 (기본값: 100px)
    """
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ 이미지 로드 실패: {image_path}")
        return []

    # 이미지 크기 가져오기
    height, width, _ = image.shape

    # 저장할 폴더 생성 (없으면 생성)
    os.makedirs(output_folder, exist_ok=True)

    count = 0
    y = 0  # 자를 위치
    base_name = os.path.splitext(os.path.basename(image_path))[0]  # 파일명 추출
    cropped_image_paths = []

    while y < height:
        # 만약 남은 높이가 crop_height보다 작다면 남은 부분만 자름
        if y + crop_height > height:
            cropped = image[y:height, 0:width]  # 남은 부분만 저장
        else:
            cropped = image[y:y+crop_height, 0:width]  # 일반적인 크롭

        # 크롭된 이미지 저장 경로
        save_path = os.path.join(output_folder, f"{base_name}_crop_{count}.jpg")
        cv2.imwrite(save_path, cropped)
        print(f"✅ 분할된 이미지 저장 완료: {save_path}")

        cropped_image_paths.append(save_path)  # OCR 수행을 위해 리스트에 추가
        count += 1

        # 다음 자를 위치를 조정 (겹치는 부분을 빼고 이동)
        y += crop_height - overlap

    print(f"📌 총 {count}개의 이미지로 분할 완료!")

    os.remove(image_path)
    
    return cropped_image_paths  # 분할된 이미지 경로 리스트 반환

In [72]:
def preprocess_image(image_path):
    """OCR 전처리를 위한 이미지 변환 및 노이즈 제거"""
    # 이미지 불러오기 (Grayscale 변환)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # 선명하게 하기 (Sharpening)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # 샤프닝 필터
    img = cv2.filter2D(img, -1, kernel)

    # 전처리된 이미지 저장 (디버깅용)
    preprocessed_path = image_path.replace(".jpg", "_processed.jpg").replace(".png", "_processed.png")
    cv2.imwrite(preprocessed_path, img)

    os.remove(image_path)

    return preprocessed_path

In [73]:
# # ✅ OCR 리더 초기화 (한국어 & 영어 지원)         # easyocr 사용 - 성능 안좋음
# reader = easyocr.Reader(['ko', 'en'], gpu=True)

# def perform_ocr_and_save(image_path):
#     """이미지에서 OCR 실행 후 결과를 텍스트 파일로 저장"""
#     try:
#         # ✅ OCR 실행
#         result = reader.readtext(image_path, detail=0)  # detail=0이면 텍스트만 추출
#         extracted_text = "\n".join(result)

#         # OCR 결과 저장 경로
#         base_name = os.path.basename(image_path).split('.')[0]  # 파일명 추출 (확장자 제거)
#         text_file_path = os.path.join(text_folder, f"{base_name}.txt")

#         # 텍스트 파일로 저장
#         with open(text_file_path, "w", encoding="utf-8") as f:
#             f.write(extracted_text)

#         print(f"📝 OCR 결과 저장 완료: {text_file_path}")

#         os.remove(image_path)

#         return text_file_path
#     except Exception as e:
#         print(f"❌ OCR 처리 중 오류 발생: {e}")
#         return None

In [78]:
# ✅ 네이버 OCR API 설정
CLOVA_OCR_URL = os.getenv("CLOVA_OCR_URL")
OCR_SECRET_KEY = os.getenv("OCR_SECRET_KEY")
HEADERS = {
    "X-OCR-SECRET": OCR_SECRET_KEY,  # 발급받은 API Key
    "Content-Type": "application/json"
}

def clova_ocr(image_path):
    """네이버 클로바 OCR API를 사용하여 이미지에서 텍스트 추출"""
    try:
        # ✅ 이미지 파일을 Base64로 인코딩
        with open(image_path, "rb") as f:
            image_data = base64.b64encode(f.read()).decode("utf-8")

        # ✅ API 요청 데이터 설정
        payload = {
            "version": "V2",
            "requestId": "sample_id",
            "timestamp": 123456789,  # 0 대신 실제 숫자로 변경
            "images": [{"format": "jpg", "name": "ocr_test", "data": image_data}]
        }

        # ✅ 네이버 OCR API 호출
        response = requests.post(CLOVA_OCR_URL, headers=HEADERS, data=json.dumps(payload))

        # ✅ API 응답을 JSON으로 변환
        result = response.json()

        # ✅ OCR 결과 추출
        extracted_text = []
        if "images" in result:
            for image in result.get("images", []):
                for field in image.get("fields", []):
                    extracted_text.append(field.get("inferText", ""))

        extracted_text_str = " ".join(extracted_text) if extracted_text else "🔹 OCR 결과 없음"

        # OCR 결과 저장 경로
        base_name = os.path.basename(image_path).split('.')[0]  # 파일명 추출 (확장자 제거)
        text_file_path = os.path.join(text_folder, f"{base_name}.txt")

        # 텍스트 파일로 저장
        with open(text_file_path, "w", encoding="utf-8") as f:
            f.write(extracted_text_str)

        print(f"📝 OCR 결과 저장 완료: {text_file_path}")

        os.remove(image_path)

        return text_file_path

    except Exception as e:
        print(f"❌ 네이버 OCR 오류 발생: {e}")
        return None

In [79]:
def merge_and_delete_text_files(text_folder, output_file):
    """여러 개의 텍스트 파일을 하나로 합친 후 기존 파일 삭제"""
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for file_name in sorted(os.listdir(text_folder)):  # 정렬된 순서로 파일 읽기
            if file_name.endswith('.txt'):  # .txt 파일만 처리
                file_path = os.path.join(text_folder, file_name)
                with open(file_path, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read())  # 파일 내용 추가
                    outfile.write("\n\n")  # 파일 구분을 위한 줄바꿈 추가
                print(f"✅ 합침: {file_name}")

    print(f"🎉 모든 텍스트 파일이 '{output_file}'로 합쳐졌습니다!")

    # ✅ 기존 txt 파일 삭제
    for file_name in os.listdir(text_folder):
        if file_name.endswith('.txt') and file_name != os.path.basename(output_file):
            file_path = os.path.join(text_folder, file_name)
            os.remove(file_path)  # 파일 삭제
            print(f"🗑 삭제 완료: {file_name}")

    print("🚀 기존 txt 파일 삭제 완료!")

In [80]:
# ✅ OCR 실행할 이미지 리스트 (이미 다운로드된 이미지 목록 가져오기)
image_files = [os.path.join(save_folder, img) for img in os.listdir(save_folder) if img.endswith((".jpg", ".png", ".jpeg"))]


for image_path in image_files:
    print(f"🚀 처리 중: {image_path}")

    # 1️⃣ 이미지 전처리
    processed_image = preprocess_image(image_path)
    if processed_image is None:
        continue  # 전처리 실패 시 건너뜀

    # 2️⃣ 전처리된 이미지 분할
    cropped_images = split_vertical_with_overlap(processed_image, cropped_folder)

    # 3️⃣ OCR 수행
    for cropped_image in cropped_images:
        # perform_ocr_and_save(cropped_image)
        clova_ocr(cropped_image)

print("🎉 모든 이미지 처리 완료!")

🚀 처리 중: download_images\image_1.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_1_processed_crop_0.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_1_processed_crop_1.jpg
📌 총 2개의 이미지로 분할 완료!
📝 OCR 결과 저장 완료: ocr_texts\image_1_processed_crop_0.txt
📝 OCR 결과 저장 완료: ocr_texts\image_1_processed_crop_1.txt
🚀 처리 중: download_images\image_2.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_2_processed_crop_0.jpg
📌 총 1개의 이미지로 분할 완료!
📝 OCR 결과 저장 완료: ocr_texts\image_2_processed_crop_0.txt
🚀 처리 중: download_images\image_3.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_0.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_1.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_2.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_3.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_4.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_5.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_6.jpg
✅ 분할된 이미지 저장 완료: cropped_images\image_3_processed_crop_7.jpg
✅ 분할된 이미지 저장 완료:

In [81]:
# ✅ 사용 예시 (폴더 경로 설정)
output_file = os.path.join(text_folder, "merged_text.txt")  # 최종 합쳐질 파일 경로

merge_and_delete_text_files(text_folder, output_file)

✅ 합침: image_1_processed_crop_0.txt
✅ 합침: image_1_processed_crop_1.txt
✅ 합침: image_2_processed_crop_0.txt
✅ 합침: image_3_processed_crop_0.txt
✅ 합침: image_3_processed_crop_1.txt
✅ 합침: image_3_processed_crop_10.txt
✅ 합침: image_3_processed_crop_11.txt
✅ 합침: image_3_processed_crop_12.txt
✅ 합침: image_3_processed_crop_13.txt
✅ 합침: image_3_processed_crop_14.txt
✅ 합침: image_3_processed_crop_15.txt
✅ 합침: image_3_processed_crop_16.txt
✅ 합침: image_3_processed_crop_17.txt
✅ 합침: image_3_processed_crop_2.txt
✅ 합침: image_3_processed_crop_3.txt
✅ 합침: image_3_processed_crop_4.txt
✅ 합침: image_3_processed_crop_5.txt
✅ 합침: image_3_processed_crop_6.txt
✅ 합침: image_3_processed_crop_7.txt
✅ 합침: image_3_processed_crop_8.txt
✅ 합침: image_3_processed_crop_9.txt
✅ 합침: image_4_processed_crop_0.txt
✅ 합침: image_4_processed_crop_1.txt
✅ 합침: image_4_processed_crop_2.txt
✅ 합침: merged_text.txt
🎉 모든 텍스트 파일이 'ocr_texts\merged_text.txt'로 합쳐졌습니다!
🗑 삭제 완료: image_1_processed_crop_0.txt
🗑 삭제 완료: image_1_processed_crop_1.txt
🗑 삭제

In [ ]:
# 환경 변수 가져오기
client = openai.OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
if client is None:
    print("🚨 OpenAI API 키가 설정되지 않았습니다! .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 정상적으로 로드되었습니다.")

✅ OpenAI API 키가 정상적으로 로드되었습니다.


In [ ]:
def correct_text_with_openai(input_text):
    """OpenAI API를 사용하여 문법, 오타, 중복 문장을 수정"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", 
                 "content":
                 """
                 당신은 문법 교정 전문가입니다. 
                 주어진 텍스트의 오타, 문법 오류, 중복 문장을 수정, 자연스럽게 바꾸세요.
                 
                 """
                 },
                {"role": "user", "content": input_text}
            ]
        )
        corrected_text = response.choices[0].message.content
        return corrected_text
    except Exception as e:
        print(f"❌ OpenAI API 오류 발생: {e}")
        return None

In [40]:
def process_text_file(input_file, output_file):
    """OCR 결과 파일을 읽고 OpenAI로 수정한 후 별도 저장"""
    try:
        # ✅ 원본 텍스트 파일 읽기
        with open(input_file, "r", encoding="utf-8") as f:
            ocr_text = f.read()

        print("🚀 OpenAI에 텍스트 전달 중...")
        corrected_text = correct_text_with_openai(ocr_text)

        if corrected_text:
            # ✅ 수정된 텍스트 별도 파일에 저장
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(corrected_text)
            print(f"✅ 수정된 텍스트 저장 완료: {output_file}")
        else:
            print("⚠️ OpenAI 응답이 없습니다.")

    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {input_file}")

In [41]:
process_text_file(input_file="ocr_texts\merged_text.txt", output_file="ocr_texts\corrected_text.txt")

🚀 OpenAI에 텍스트 전달 중...
✅ 수정된 텍스트 저장 완료: ocr_texts\corrected_text.txt
